In [ ]:
import pandas as pd
import json
import os
import pgeocode

# Adjust this path if you move the CSV, but this should work from the notebook's folder
csv_path = "fake_users.csv"

# Path for JSON progress file (index -> {lat, lon})
json_progress_path = "fake_users_geo_progress.json"

# Load the CSV
users_df = pd.read_csv(csv_path)
print("Loaded rows:", len(users_df))
print(users_df[["postal_code"]].head())

# Make sure postal_code is a string
users_df["postal_code"] = users_df["postal_code"].astype(str).str.zfill(5)

# Load any existing progress so we can resume
if os.path.exists(json_progress_path):
    with open(json_progress_path, "r") as f:
        progress = json.load(f)
    print(f"Loaded existing progress for {len(progress)} rows from {json_progress_path}")
else:
    progress = {}
    print("No existing progress file found; starting fresh.")

# Offline ZIP-code lookup for US
nomi = pgeocode.Nominatim("us")

# Prepare columns for results
users_df["latitude"] = None
users_df["longitude"] = None

# If we have existing progress, pre-fill the DataFrame
for idx_str, coords in progress.items():
    idx = int(idx_str)
    users_df.at[idx, "latitude"] = coords.get("lat")
    users_df.at[idx, "longitude"] = coords.get("lon")

# Main loop: use ZIP only (no external API)
for idx, row in users_df.iterrows():
    idx_str = str(idx)

    # Skip if we already have coordinates for this index
    if idx_str in progress:
        continue

    zip_code = row["postal_code"]

    try:
        info = nomi.query_postal_code(zip_code)
        if info is not None and not pd.isna(info.latitude) and not pd.isna(info.longitude):
            lat = float(info.latitude)
            lon = float(info.longitude)
        else:
            lat = None
            lon = None
    except Exception as e:
        print(f"Error looking up ZIP {zip_code} at index {idx}: {e}")
        lat = None
        lon = None

    # Update DataFrame
    users_df.at[idx, "latitude"] = lat
    users_df.at[idx, "longitude"] = lon

    # Update in-memory progress dict and write it to disk so we can resume later
    progress[idx_str] = {"lat": lat, "lon": lon}
    with open(json_progress_path, "w") as f:
        json.dump(progress, f)

    # Optional: progress logging
    if idx % 10 == 0:
        print(f"Processed {idx+1} / {len(users_df)} ZIP codes; progress saved to {json_progress_path}")

# Save results to a new CSV so we don’t overwrite the original
output_path = "fake_users_with_geo.csv"
users_df.to_csv(output_path, index=False)
print(f"Saved ZIP-based coordinates to {output_path}")
print(f"Progress JSON saved to {json_progress_path}")



Loaded rows: 238
       username  password first_name          last_name             address1  \
0   Jestrada491  CPSC491!       Jake            Estrada    1030 Calle Sombra   
1      Htran491  CPSC491!     Hubert               Tran  27572 Sweetbrier ln   
2      Ktran491  CPSC491!    Kenneth               Tran          22 La Ronda   
3  EMartinez491  CPSC491!    Eduardo  Salvador-Martinez  52 Carriage Hill Ln   
4      rudy_312    xcr666      Genia            Watrous   1818 State Route 3   

            city state  postal_code                            email  \
0   San Clemente    CA        92672    Jestrada309@csu.fullerton.edu   
1  Mission Viejo    CA        92691     huberttranstemclub@gmail.com   
2         Irvine    CA        92606        kqmtran@csu.fullerton.edu   
3   Laguna Hills    CA        92653  eduardosal714@csu.fullerton.edu   
4         Fulton    NY        13069               rudy.312@yahoo.com   

          phone date_of_birth                  created_at      role  

RateLimiter caught an error, retrying (0/2 tries). Called with (*('1030 Calle Sombra, San Clemente, CA 92672',), **{}).
Traceback (most recent call last):
  File "/home/jake/.local/lib/python3.10/site-packages/urllib3/connectionpool.py", line 534, in _make_request
    response = conn.getresponse()
  File "/home/jake/.local/lib/python3.10/site-packages/urllib3/connection.py", line 565, in getresponse
    httplib_response = super().getresponse()
  File "/usr/lib/python3.10/http/client.py", line 1375, in getresponse
    response.begin()
  File "/usr/lib/python3.10/http/client.py", line 318, in begin
    version, status, reason = self._read_status()
  File "/usr/lib/python3.10/http/client.py", line 279, in _read_status
    line = str(self.fp.readline(_MAXLINE + 1), "iso-8859-1")
  File "/usr/lib/python3.10/socket.py", line 705, in readinto
    return self._sock.recv_into(b)
  File "/usr/lib/python3.10/ssl.py", line 1303, in recv_into
    return self.read(nbytes, buffer)
  File "/usr/lib/py

Processed 1 / 238 addresses; progress saved to fake_users_geo_progress.json


RateLimiter caught an error, retrying (0/2 tries). Called with (*('27572 Sweetbrier ln, Mission Viejo, CA 92691',), **{}).
Traceback (most recent call last):
  File "/home/jake/.local/lib/python3.10/site-packages/urllib3/connectionpool.py", line 534, in _make_request
    response = conn.getresponse()
  File "/home/jake/.local/lib/python3.10/site-packages/urllib3/connection.py", line 565, in getresponse
    httplib_response = super().getresponse()
  File "/usr/lib/python3.10/http/client.py", line 1375, in getresponse
    response.begin()
  File "/usr/lib/python3.10/http/client.py", line 318, in begin
    version, status, reason = self._read_status()
  File "/usr/lib/python3.10/http/client.py", line 279, in _read_status
    line = str(self.fp.readline(_MAXLINE + 1), "iso-8859-1")
  File "/usr/lib/python3.10/socket.py", line 705, in readinto
    return self._sock.recv_into(b)
  File "/usr/lib/python3.10/ssl.py", line 1303, in recv_into
    return self.read(nbytes, buffer)
  File "/usr/lib